In [1]:
%cd ../

In [2]:
import logging
import numpy as np
import htfa_torch.dtfa as DTFA
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils
import htfa_torch.htfa as HTFA


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/utils/deprecation.py

In [26]:
from scipy.spatial import distance
from scipy import stats
import nilearn.plotting as niplot
import statsmodels.stats.multitest as smt
from ordered_set import OrderedSet
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score,StratifiedKFold
from sklearn.feature_selection import SelectPercentile, f_classif,SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.metrics import plot_roc_curve

In [4]:
import matplotlib
import matplotlib.pyplot as pl
from matplotlib import cycler
from seaborn.apionly import color_palette
column_width = 3.3
page_width = 6.7
# colorblind-friendly colors 
# source: https://personal.sron.nl/~pault/
colors = {
    'bright': ['4477AA', 
               '66CCEE', 
               '228833', 
               'CCBB44', 
               'EE6677', 
               'AA3377', 
               'BBBBBB'],
    'high_contrast': ['FFFFFF', 
                      'DDAA33', 
                      'BB5566', 
                      '004488', 
                      '00000'],
    'vibrant': ['0077BB', 
                '33BBEE', 
                '0099BB',
                'EE7733',
                'CC3311',
                'EE3377',
                'BBBBBB'],
    'muted': ['332288',
              '88CCEE',
              '44AA99',
              '117733',
              '999933',
              'DDCC77',
              'CC6677',
              '882255',
              'AA4499']
}
pl.rc('legend', frameon=False)
pl.rc('figure', figsize=(column_width, 
                         0.66 * column_width),
                dpi=120)
pl.rc('savefig', dpi=300)
pl.rc('font', size=8)


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/matplotlib/__init__.py:855: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/matplotlib/__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/seaborn/apionly.py:9: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [5]:
%matplotlib inline

In [6]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [7]:
affvids_db = niidb.FMriActivationsDb('data/affvids2018_mc_zrest.db')

In [8]:
htfa = HTFA.HierarchicalTopographicFactorAnalysis([b for b in affvids_db.all() if 'rest' not in b.task], mask='/data/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz', num_factors=100)

02/03/2020 13:47:39 Loading Nifti image /data/zulqarnain/AffVids_mcwm/motion_corrected/sub10_run02_mc_MNI_masked.nii.gz with mask /data/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None
02/03/2020 13:48:00 Loading Nifti image /data/zulqarnain/AffVids_mcwm/motion_corrected/sub10_run03_mc_MNI_masked.nii.gz with mask /data/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed i

02/03/2020 13:53:00 Loading Nifti image /data/zulqarnain/AffVids_mcwm/motion_corrected/sub15_run02_mc_MNI_masked.nii.gz with mask /data/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None
02/03/2020 13:53:21 Loading Nifti image /data/zulqarnain/AffVids_mcwm/motion_corrected/sub15_run03_mc_MNI_masked.nii.gz with mask /data/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed i

02/03/2020 13:58:29 Loading Nifti image /data/zulqarnain/AffVids_mcwm/motion_corrected/sub23_run02_mc_MNI_masked.nii.gz with mask /data/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None
02/03/2020 13:58:51 Loading Nifti image /data/zulqarnain/AffVids_mcwm/motion_corrected/sub25_run01_mc_MNI_masked.nii.gz with mask /data/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed i

02/03/2020 14:03:58 Loading Nifti image /data/zulqarnain/AffVids_mcwm/motion_corrected/sub05_run01_mc_MNI_masked.nii.gz with mask /data/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None
02/03/2020 14:04:20 Loading Nifti image /data/zulqarnain/AffVids_mcwm/motion_corrected/sub05_run02_mc_MNI_masked.nii.gz with mask /data/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed i

In [9]:
htfa.load_state('data/affvids_norest_k100_n1000_01242020_230748')

In [10]:
validation_filter = affvids_db.inference_filter(training=False)
validation_blocks = [b for (b, block) in enumerate(htfa._blocks) if validation_filter(block)]

In [11]:
def task_labeler(task):
    if 'heights' in task:
        return 'Heights'
    elif 'social' in task:
        return 'Social threat'
    elif 'spider' in task:
        return 'Spiders'
    elif 'rest' in task:
        return 'Rest'
    else:
        return None

def rest_task_labeler(task):
    if task == 'rest':
        return 'Rest'
    else:
        return 'Task'

def high_low_labeler(task):
    if 'high' in task:
        return 'high'
    elif 'low' in task:
        return 'low'
    else:
        return None

In [12]:
def block_task_labeler(block):
    return task_labeler(block.task)

def block_high_low_labeler(block):
    if 'high' in block.task:
        return 'high'
    elif 'low' in block.task:
        return 'low'
    else:
        return None

In [13]:
def subject_labeler(subject):
    return 'Subject %d' % subject

In [14]:
def block_subject_labeler(block):
    return subject_labeler(block.subject)

In [27]:
def classification_pipeline(data,targets,session_labels,cv=LeaveOneGroupOut(),
                                 scoring="roc_auc",feature_select=False,
                            feature_selection=SelectPercentile(f_classif, percentile=5),print_all=True):
    categories = np.unique(targets)
    overall_result = []
    overall_result_chance = []
    category_result = {key:None for key in categories}
    for category in categories:
        print("Processing %s" % (category))
        dummy_classifier = DummyClassifier()
        classification_target = [t==category for t in targets]
        svc = SVC(C=1., kernel="linear",probability=True,random_state=42)
        if feature_select:
            classification_target = np.array(classification_target)
            result = []
            result_chance = []
            for train_ind, test_ind in cv.split(data,classification_target,groups=session_labels):
                print("TRAIN:", train_ind, "TEST:", test_ind)
                train_ind = np.array(train_ind).astype(int)
                test_ind = np.array(test_ind).astype(int)
                training_labels = classification_target[train_ind]
                testing_labels = classification_target[test_ind]
                train_feature_classif = feature_selection.fit(data[train_ind],training_labels)
                training_data = train_feature_classif.transform(data[train_ind])
                testing_data = train_feature_classif.transform(data[test_ind])

                train_svc = svc.fit(training_data,training_labels)
                train_dummy = dummy_classifier.fit(training_data,training_labels)
                viz = plot_roc_curve(train_svc,testing_data,testing_labels)
                result.append(viz.roc_auc)
                pl.close()
                viz = plot_roc_curve(train_dummy,testing_data,testing_labels)
                result_chance.append(viz.roc_auc)
                pl.close()
        else:
            classifier = svc
            result = cross_val_score(
                classifier,
                data,
                classification_target,
                cv=cv,
                groups=session_labels,
                scoring="roc_auc",
            )

            result_chance = cross_val_score(
                dummy_classifier,
                data,
                classification_target,
                cv=cv,
                groups=session_labels,
                scoring="roc_auc",
            )
        result = np.array(result)
        result_chance = np.array(result_chance)
        category_result[category] = result
        overall_result.extend(result)
        overall_result_chance.extend(result_chance)
        if print_all:
            print("Scores: %1.4f +- %1.4f" % (
                result.mean(),
                result.std()))

            print("Chance Scores: %1.4f +- %1.4f" % (
                result_chance.mean(),
                result_chance.std()))
    
    if print_all:
        print ('...')
        print("Overall Score: %1.4f +- %1.4f" % (
        np.array(overall_result).mean(),
        np.array(overall_result).std()))

        print("Overall Dummy Score: %1.4f +- %1.4f" % (
        np.array(overall_result_chance).mean(),
        np.array(overall_result_chance).std()))

        print ("\n")
    return overall_result,overall_result_chance,category_result

In [16]:
all_weights_htfa = htfa.enc.hyperparams.block__weights__mu.data.numpy()
confidence = 0.95

In [28]:
### task classification using leave one out runs on HTFA###
cv = LeaveOneGroupOut()
summary_result = []
summary_result_chance = []
summary_result_category = []
summary_subject_means = []
subjects = OrderedSet([b.subject for b in htfa._blocks])
for s in subjects:
    print("Processing Subject %d" % (s))
    session_labels = []
    data = []
    targets = []
    for i in range(htfa.num_blocks):
        if i not in validation_blocks:
            block = htfa._blocks[i]
            if block.subject == s:
                session_labels.append(block.run)
                data.append(all_weights_htfa[i,:,:].mean(0))
                targets.append(block_task_labeler(block))
    data = np.array(data)
    overall_result,overall_result_chance,category_result = classification_pipeline(data=data,
                                                                   targets=targets,
                                                                 session_labels = session_labels,
                                                                   scoring="roc_auc",
                                                                   print_all=True)
    summary_result_category.append(category_result)
    summary_result.extend(overall_result)
    summary_result_chance.extend(overall_result_chance)
    summary_subject_means.append(np.array(overall_result).mean())

print ('...')
category_result = {key:[] for key in np.unique(targets)}
for c in np.unique(targets):
    for s in summary_result_category:
        category_result[c].extend(s[c])
    print("HTFA Summary Score Across Subjects, For Category %s : %1.4f +- %1.4f" % (c,
np.array(category_result[c]).mean(),
np.array(category_result[c]).std()))
    
print ('...')
print("HTFA Summary Score Across Subjects, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_result).mean(),
np.array(summary_result).std()))

print("Overall Dummy Score: %1.2f +- %1.2f" % (
np.array(summary_result_chance).mean(),
np.array(summary_result_chance).std()))

print ('...')
print("HTFA Summary Mean/CI, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_subject_means).mean(),
stats.sem(summary_subject_means) 
    * stats.t.ppf((1 + confidence) / 2, len(summary_subject_means) - 1)))

htfa_summary_result = summary_result
htfa_summary_means = summary_subject_means

Processing Subject 10
Processing Heights
Scores: 0.9271 +- 0.0390
Chance Scores: 0.4583 +- 0.1284
Processing Social threat
Scores: 0.9375 +- 0.0442
Chance Scores: 0.4792 +- 0.0295
Processing Spiders
Scores: 0.9479 +- 0.0390
Chance Scores: 0.3958 +- 0.0589
...
Overall Score: 0.9375 +- 0.0417
Overall Dummy Score: 0.4444 +- 0.0905


Processing Subject 11
Processing Heights
Scores: 0.6622 +- 0.0406
Chance Scores: 0.7202 +- 0.1104
Processing Social threat
Scores: 0.8438 +- 0.0765
Chance Scores: 0.4583 +- 0.1179
Processing Spiders
Scores: 0.6910 +- 0.1756
Chance Scores: 0.4514 +- 0.0687
...
Overall Score: 0.7323 +- 0.1383
Overall Dummy Score: 0.5433 +- 0.1610


Processing Subject 12
Processing Heights
Scores: 0.3819 +- 0.2172
Chance Scores: 0.5556 +- 0.0520
Processing Social threat
Scores: 0.6057 +- 0.0807
Chance Scores: 0.3631 +- 0.0537
Processing Spiders
Scores: 0.5060 +- 0.1094
Chance Scores: 0.5119 +- 0.0537
...
Overall Score: 0.4979 +- 0.1740
Overall Dummy Score: 0.4769 +- 0.0980


Proc

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sk

Scores: 0.8611 +- 0.0322
Chance Scores: 0.3542 +- 0.0295
...
Overall Score: 0.8446 +- 0.0965
Overall Dummy Score: 0.4325 +- 0.1259


Processing Subject 14
Processing Heights
Scores: 0.8750 +- 0.0312
Chance Scores: 0.4688 +- 0.0312
Processing Social threat
Scores: 0.9375 +- 0.0312
Chance Scores: 0.4688 +- 0.0938
Processing Spiders
Scores: 0.9844 +- 0.0156
Chance Scores: 0.5938 +- 0.1562
...
Overall Score: 0.9323 +- 0.0523
Overall Dummy Score: 0.5104 +- 0.1219


Processing Subject 15
Processing Heights
Scores: 0.6007 +- 0.1077
Chance Scores: 0.4375 +- 0.0884
Processing Social threat
Scores: 0.8557 +- 0.0807
Chance Scores: 0.5774 +- 0.0673
Processing Spiders
Scores: 0.8318 +- 0.1294
Chance Scores: 0.5179 +- 0.1246
...
Overall Score: 0.7627 +- 0.1576
Overall Dummy Score: 0.5109 +- 0.1121


Processing Subject 16
Processing Heights
Scores: 0.8472 +- 0.1094
Chance Scores: 0.5139 +- 0.0196
Processing Social threat
Scores: 1.0000 +- 0.0000
Chance Scores: 0.3988 +- 0.1431
Processing Spiders
Scor

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sk

Scores: 0.8854 +- 0.0390
Chance Scores: 0.3056 +- 0.0786
Processing Social threat
Scores: 0.8140 +- 0.0432
Chance Scores: 0.6875 +- 0.0510
Processing Spiders
Scores: 0.6741 +- 0.1404
Chance Scores: 0.6429 +- 0.1051
...
Overall Score: 0.7912 +- 0.1241
Overall Dummy Score: 0.5453 +- 0.1889


Processing Subject 23
Processing Heights
Scores: 0.5848 +- 0.0223
Chance Scores: 0.6250 +- 0.1250
Processing Social threat
Scores: 0.6607 +- 0.0893
Chance Scores: 0.3304 +- 0.0446
Processing Spiders
Scores: 0.7552 +- 0.0885
Chance Scores: 0.5208 +- 0.0208
...
Overall Score: 0.6669 +- 0.1015
Overall Dummy Score: 0.4921 +- 0.1446


Processing Subject 25
Processing Heights
Scores: 0.6146 +- 0.1206
Chance Scores: 0.3661 +- 0.0406
Processing Social threat
Scores: 0.9410 +- 0.0214
Chance Scores: 0.5000 +- 0.0510
Processing Spiders
Scores: 0.9033 +- 0.0422
Chance Scores: 0.5804 +- 0.0789
...
Overall Score: 0.8196 +- 0.1639
Overall Dummy Score: 0.4821 +- 0.1063


Processing Subject 26
Processing Heights
Scor

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sk

Scores: 0.8408 +- 0.0729
Chance Scores: 0.4762 +- 0.0547
Processing Social threat
Scores: 0.8958 +- 0.1031
Chance Scores: 0.5744 +- 0.0661
Processing Spiders
Scores: 0.8576 +- 0.0603
Chance Scores: 0.5556 +- 0.0839
...
Overall Score: 0.8647 +- 0.0840
Overall Dummy Score: 0.5354 +- 0.0813


Processing Subject 5
Processing Heights
Scores: 0.5818 +- 0.1012
Chance Scores: 0.5149 +- 0.1433
Processing Social threat
Scores: 0.6176 +- 0.1696
Chance Scores: 0.4702 +- 0.1588
Processing Spiders
Scores: 0.7569 +- 0.1513
Chance Scores: 0.4583 +- 0.0295
...
Overall Score: 0.6521 +- 0.1623
Overall Dummy Score: 0.4812 +- 0.1270


Processing Subject 6
Processing Heights
Scores: 0.2976 +- 0.1443
Chance Scores: 0.5208 +- 0.1062
Processing Social threat
Scores: 0.3527 +- 0.1325
Chance Scores: 0.4375 +- 0.1350
Processing Spiders
Scores: 0.5799 +- 0.1464
Chance Scores: 0.4306 +- 0.0937
...
Overall Score: 0.4101 +- 0.1867
Overall Dummy Score: 0.4630 +- 0.1202


Processing Subject 7
Processing Heights
Scores:

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sk

In [20]:
from scipy.stats import pearsonr
from scipy.special import betainc
def corrcoef(matrix):
    r = np.corrcoef(matrix)
    rf = r[np.triu_indices(r.shape[0], 1)]
    df = matrix.shape[1] - 2
    ts = rf * rf * (df / (1 - rf * rf))
    pf = betainc(0.5 * df, 0.5, df / (df + ts))
    p = np.zeros(shape=r.shape)
    p[np.triu_indices(p.shape[0], 1)] = pf
    p[np.tril_indices(p.shape[0], -1)] = p.T[np.tril_indices(p.shape[0], -1)]
    p[np.diag_indices(p.shape[0])] = np.ones(p.shape[0])
    return r, p


In [29]:
### functional connectivity using HTFA ###

spider_matrix = []
social_matrix = []
heights_matrix = []
all_matrix = []

spider_matrix_p = []
social_matrix_p = []
heights_matrix_p = []
all_matrix_p = []
for i in range(htfa.num_blocks):
    r,p = corrcoef(all_weights_htfa[i,:,:].T)
    p = p < 0.05
    all_matrix.append(r)
    all_matrix_p.append(p)
    if block_task_labeler(htfa._blocks[i]) == 'Spiders':
        r,p = corrcoef(all_weights_htfa[i,:,:].T)
        p = p < 0.05
        spider_matrix.append(r)
        spider_matrix_p.append(p)
    elif  block_task_labeler(htfa._blocks[i]) == 'Social threat':
        r,p = corrcoef(all_weights_htfa[i,:,:].T)
        p = p < 0.05
        social_matrix.append(r)
        social_matrix_p.append(p)
    elif  block_task_labeler(htfa._blocks[i]) == 'Heights':
        r,p = corrcoef(all_weights_htfa[i,:,:].T)
        p = p < 0.05
        heights_matrix.append(r)
        heights_matrix_p.append(p)

        
        
spider_matrix = (np.array(spider_matrix))
social_matrix = (np.array(social_matrix))
heights_matrix = (np.array(heights_matrix))
all_matrix = np.array(all_matrix)

spider_matrix_p = (np.array(spider_matrix_p))
social_matrix_p = (np.array(social_matrix_p))
heights_matrix_p = (np.array(heights_matrix_p))
all_matrix_p = np.array(all_matrix_p)

spider_matrix = spider_matrix * spider_matrix_p
social_matrix = social_matrix * social_matrix_p
heights_matrix = heights_matrix * heights_matrix_p
all_matrix = all_matrix * all_matrix_p

all_vectors = np.zeros(shape=(all_matrix.shape[0],4950))
all_vectors_p = np.zeros(shape=(all_matrix_p.shape[0],4950))

for i in range(all_matrix.shape[0]):
    all_vectors[i,:] = distance.squareform(all_matrix[i,:,:],checks=False)
    all_vectors_p[i,:] = distance.squareform(all_matrix_p[i,:,:],checks=False)

spider_vectors = np.zeros(shape=(spider_matrix.shape[0],4950))
spider_vectors_p = np.zeros(shape=(spider_matrix_p.shape[0],4950))

for i in range(spider_matrix.shape[0]):
    spider_vectors[i,:] = distance.squareform(spider_matrix[i,:,:],checks=False)
    spider_vectors_p[i,:] = distance.squareform(spider_matrix_p[i,:,:],checks=False)


social_vectors = np.zeros(shape=(social_matrix.shape[0],4950))
social_vectors_p = np.zeros(shape=(social_matrix_p.shape[0],4950))

for i in range(social_matrix.shape[0]):
    social_vectors[i,:] = distance.squareform(social_matrix[i,:,:],checks=False)
    social_vectors_p[i,:] = distance.squareform(social_matrix_p[i,:,:],checks=False)

    
heights_vectors = np.zeros(shape=(heights_matrix.shape[0],4950))
heights_vectors_p = np.zeros(shape=(heights_matrix_p.shape[0],4950))

for i in range(heights_matrix.shape[0]):
    heights_vectors[i,:] = distance.squareform(heights_matrix[i,:,:],checks=False)
    heights_vectors_p[i,:] = distance.squareform(heights_matrix_p[i,:,:],checks=False)
    
spider_vectors_mean = spider_vectors.mean(0)
spider_vectors_std = spider_vectors.std(0)
social_vectors_mean = social_vectors.mean(0)
social_vectors_std = social_vectors.std(0)
heights_vectors_mean = heights_vectors.mean(0)
heights_vectors_std = heights_vectors.std(0)

In [30]:
### task classification using leave one out runs on HTFA using functional connectivity###
cv = LeaveOneGroupOut()
summary_result = []
summary_result_chance = []
summary_result_category = []
summary_subject_means = []

subjects = OrderedSet([b.subject for b in htfa._blocks])

for s in subjects:
    print("Processing Subject %d" % (s))
    session_labels = []
    data = []
    targets = []
    for i in range(htfa.num_blocks):
        if i not in validation_blocks:
            block = htfa._blocks[i]
            if block.subject == s:
                session_labels.append(block.run)
                data.append(all_vectors[i])
                targets.append(block_task_labeler(block))
    data = np.array(data)
    overall_result,overall_result_chance,category_result = classification_pipeline(data=data,
                                                                   targets=targets,
                                                                session_labels = session_labels,
                                                                   scoring="roc_auc",
                                                                   print_all=True)
    summary_result_category.append(category_result)
    summary_result.extend(overall_result)
    summary_result_chance.extend(overall_result_chance)
    summary_subject_means.append(np.array(overall_result).mean())

print ('...')
category_result = {key:[] for key in np.unique(targets)}
for c in np.unique(targets):
    for s in summary_result_category:
        category_result[c].extend(s[c])
    print("HTFA Summary Score Across Subjects, For Category %s : %1.4f +- %1.4f" % (c,
np.array(category_result[c]).mean(),
np.array(category_result[c]).std()))
    
print ('...')
print("HTFA Summary Score Across Subjects, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_result).mean(),
np.array(summary_result).std()))

print ('...')
print("HTFA Summary Mean/CI, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_subject_means).mean(),
stats.sem(summary_subject_means) 
    * stats.t.ppf((1 + confidence) / 2, len(summary_subject_means) - 1)))

htfa_summary_result = summary_result
dummy_summary_result = summary_result_chance
htfa_summary_means = summary_subject_means


Processing Subject 10
Processing Heights
Scores: 0.8542 +- 0.0966
Chance Scores: 0.3958 +- 0.0780
Processing Social threat
Scores: 0.9479 +- 0.0390
Chance Scores: 0.4167 +- 0.0295
Processing Spiders


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sk

Scores: 0.8021 +- 0.0966
Chance Scores: 0.6250 +- 0.1840
...
Overall Score: 0.8681 +- 0.1018
Overall Dummy Score: 0.4792 +- 0.1559


Processing Subject 11
Processing Heights
Scores: 0.5536 +- 0.0526
Chance Scores: 0.4673 +- 0.1807
Processing Social threat
Scores: 0.8080 +- 0.1295
Chance Scores: 0.5714 +- 0.1641
Processing Spiders


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sk

Scores: 0.6424 +- 0.1449
Chance Scores: 0.4653 +- 0.1525
...
Overall Score: 0.6680 +- 0.1570
Overall Dummy Score: 0.5013 +- 0.1734


Processing Subject 12
Processing Heights
Scores: 0.3368 +- 0.1663
Chance Scores: 0.6597 +- 0.0873
Processing Social threat
Scores: 0.3720 +- 0.1096
Chance Scores: 0.4435 +- 0.1459
Processing Spiders


...

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sk

Scores: 0.4479 +- 0.0966
Chance Scores: 0.5972 +- 0.1094
...
Overall Score: 0.3814 +- 0.1323
Overall Dummy Score: 0.5066 +- 0.1112


Processing Subject 7
Processing Heights
Scores: 0.5312 +- 0.0000
Chance Scores: 0.3438 +- 0.0938
Processing Social threat
Scores: 0.4688 +- 0.0312
Chance Scores: 0.5625 +- 0.0000
Processing Spiders
Scores: 0.4688 +- 0.0312
Chance Scores: 0.4375 +- 0.0000
...
Overall Score: 0.4896 +- 0.0390
Overall Dummy Score: 0.4479 +- 0.1047


Processing Subject 8
Processing Heights
Scores: 0.8869 +- 0.0879
Chance Scores: 0.4048 +- 0.1121
Processing Social threat


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sk

Scores: 0.8785 +- 0.0299
Chance Scores: 0.4236 +- 0.1874
Processing Spiders
Scores: 0.8720 +- 0.1337
Chance Scores: 0.5774 +- 0.0845
...
Overall Score: 0.8791 +- 0.0942
Overall Dummy Score: 0.4686 +- 0.1557


Processing Subject 9
Processing Heights
Scores: 0.4092 +- 0.1154
Chance Scores: 0.5536 +- 0.1010
Processing Social threat
Scores: 0.5938 +- 0.1169
Chance Scores: 0.5714 +- 0.1719
Processing Spiders
Scores: 0.6458 +- 0.0780
Chance Scores: 0.6458 +- 0.1640
...
Overall Score: 0.5496 +- 0.1460
Overall Dummy Score: 0.5903 +- 0.1543


...
HTFA Summary Score Across Subjects, For Category Heights : 0.6860 +- 0.2104
HTFA Summary Score Across Subjects, For Category Social threat : 0.7067 +- 0.2102
HTFA Summary Score Across Subjects, For Category Spiders : 0.6871 +- 0.2044
...
HTFA Summary Score Across Subjects, Across Tasks: 0.6933 +- 0.2086
...
HTFA Summary Mean/CI, Across Tasks: 0.6908 +- 0.0709


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sk

In [31]:
### task classification using leave one out runs on HTFA using functional connectivity with feature selection###
cv = LeaveOneGroupOut()
summary_result = []
summary_result_chance = []
summary_result_category = []
summary_subject_means = []

subjects = OrderedSet([b.subject for b in htfa._blocks])
feature_selection = SelectKBest(f_classif, k=500)

for s in subjects:
    print("Processing Subject %d" % (s))
    session_labels = []
    data = []
    targets = []
    for i in range(htfa.num_blocks):
        if i not in validation_blocks:
            block = htfa._blocks[i]
            if block.subject == s:
                session_labels.append(block.run)
                data.append(np.arctanh(all_vectors[i]))
                targets.append(block_task_labeler(block))
    data = np.array(data)
    overall_result,overall_result_chance,category_result = classification_pipeline(data=data,
                                                                   targets=targets,
                                                                session_labels = session_labels,
                                                                   scoring="roc_auc",feature_select=True,
                                                                   feature_selection=feature_selection,
                                                                   print_all=True)
    summary_result_category.append(category_result)
    summary_result.extend(overall_result)
    summary_result_chance.extend(overall_result_chance)
    summary_subject_means.append(np.array(overall_result).mean())

print ('...')
category_result = {key:[] for key in np.unique(targets)}
for c in np.unique(targets):
    for s in summary_result_category:
        category_result[c].extend(s[c])
    print("HTFA Summary Score Across Subjects, For Category %s : %1.4f +- %1.4f" % (c,
np.array(category_result[c]).mean(),
np.array(category_result[c]).std()))
    
print ('...')
print("HTFA Summary Score Across Subjects, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_result).mean(),
np.array(summary_result).std()))

print ('...')
print("HTFA Summary Mean/CI, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_subject_means).mean(),
stats.sem(summary_subject_means) 
    * stats.t.ppf((1 + confidence) / 2, len(summary_subject_means) - 1)))

htfa_summary_result = summary_result
dummy_summary_result = summary_result_chance
htfa_summary_means = summary_subject_means


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [   9  110  125  138  139  150  163  165  174  176  187  198  206  211
  221  225  227  233  241  242  248  252  258  262  266  268  274  276
  280  288  289  290  371  373  374  376  407  465  469  580  582  631
  648  651  653  672  687  709  740  764  767  770  779  782  789  804
  808  809  830  832  841  844  945  947  953  960  970  971  974  977
  982  984  986  989 1001 1011 1013 1014 1016 1017 1019 1020 1021 1022
 1023 1024 1026 1027 1031 1055 1078 1098 1102 1105 1137 1141 1148 1154
 1156 1158 1159 1168 1170 1178 1181 1182 1185 1187 1190 1193 1194 1198
 1200 1202 1205 1207 1228 1231 1274 1325 1359 1443 1453 1469 1470 1523
 1547 1615 1666 1690 1883 1949 1954 1956 1969 1978 1982 1997 2028 2033
 2043 2056 2061 2067 2083 2085 2089 2090 2094 2096 2097 2225 2370 2373
 2374 2377 2444 2537 2543 2557 2571 2575 2580 2585 2648 2715 2718 2745


Processing Subject 10
Processing Heights
TRAIN: [ 0  1  3  4  6  7  9 11 12 14 15 17 18 20 21 23 24 26 27 29 30 32 33 34] TEST: [ 2  5  8 10 13 16 19 22 25 28 31 35]
TRAIN: [ 1  2  3  5  6  8  9 10 12 13 15 16 18 19 21 22 24 25 27 28 30 31 33 35] TEST: [ 0  4  7 11 14 17 20 23 26 29 32 34]
TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 22 23 25 26 28 29 31 32 34 35] TEST: [ 1  3  6  9 12 15 18 21 24 27 30 33]
Scores: 0.7292 +- 0.0966
Chance Scores: 0.5208 +- 0.1062
Processing Social threat
TRAIN: [ 0  1  3  4  6  7  9 11 12 14 15 17 18 20 21 23 24 26 27 29 30 32 33 34] TEST: [ 2  5  8 10 13 16 19 22 25 28 31 35]
TRAIN: [ 1  2  3  5  6  8  9 10 12 13 15 16 18 19 21 22 24 25 27 28 30 31 33 35] TEST: [ 0  4  7 11 14 17 20 23 26 29 32 34]


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [   9  102  103  120  140  141  151  176  181  186  196  204  206  210
  233  237  238  240  244  249  252  264  266  269  270  286  287  288
  290  293  313  354  361  371  374  376  407  465  552  563  584  629
  648  651  672  674  767  770  781  793  795  797  801  802  804  807
  814  816  823  828  834  841  857  921  945  947  952  953  957  960
  965  966  978  989  990  993  996 1005 1010 1017 1019 1023 1024 1025
 1026 1027 1031 1033 1055 1067 1078 1105 1122 1123 1124 1127 1137 1139
 1140 1141 1142 1147 1154 1157 1158 1159 1163 1168 1181 1185 1186 1189
 1191 1192 1193 1200 1207 1229 1274 1359 1361 1389 1444 1453 1469 1527
 1640 1682 1690 1802 1849 1853 1949 1954 1956 1960 1963 1965 1969 1970
 1974 1979 1982 1997 1998 2016 2028 2033 2036 2045 2056 2061 2072 2073
 2079 2085 2088 2094 2097 2105 2155 2220 2225 2268 2401 2410 2444 2508


TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 22 23 25 26 28 29 31 32 34 35] TEST: [ 1  3  6  9 12 15 18 21 24 27 30 33]
Scores: 0.9271 +- 0.0531
Chance Scores: 0.6042 +- 0.1062
Processing Spiders
TRAIN: [ 0  1  3  4  6  7  9 11 12 14 15 17 18 20 21 23 24 26 27 29 30 32 33 34] TEST: [ 2  5  8 10 13 16 19 22 25 28 31 35]
TRAIN: [ 1  2  3  5  6  8  9 10 12 13 15 16 18 19 21 22 24 25 27 28 30 31 33 35] TEST: [ 0  4  7 11 14 17 20 23 26 29 32 34]


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [   9  102  103  120  140  141  151  176  181  186  196  204  206  210
  233  237  238  240  244  249  252  264  266  269  270  286  287  288
  290  293  313  354  361  371  374  376  407  465  552  563  584  629
  648  651  672  674  767  770  781  793  795  797  801  802  804  807
  814  816  823  828  834  841  857  921  945  947  952  953  957  960
  965  966  978  989  990  993  996 1005 1010 1017 1019 1023 1024 1025
 1026 1027 1031 1033 1055 1067 1078 1105 1122 1123 1124 1127 1137 1139
 1140 1141 1142 1147 1154 1157 1158 1159 1163 1168 1181 1185 1186 1189
 1191 1192 1193 1200 1207 1229 1274 1359 1361 1389 1444 1453 1469 1527
 1640 1682 1690 1802 1849 1853 1949 1954 1956 1960 1963 1965 1969 1970
 1974 1979 1982 1997 1998 2016 2028 2033 2036 2045 2056 2061 2072 2073
 2079 2085 2088 2094 2097 2105 2155 2220 2225 2268 2401 2410 2444 2508


...

TRAIN: [ 0  2  3  5  6  8  9 11 12 14 15 17 18 20 21 23 24 26 27 29 30 31 32 34] TEST: [ 1  4  7 10 13 16 19 22 25 28 33]
TRAIN: [ 0  1  3  4  6  7  9 10 12 13 15 16 18 19 21 22 24 25 27 28 30 32 33] TEST: [ 2  5  8 11 14 17 20 23 26 29 31 34]
Scores: 0.8036 +- 0.1536
Chance Scores: 0.4702 +- 0.0463
...
Overall Score: 0.8532 +- 0.1053
Overall Dummy Score: 0.4838 +- 0.1150


Processing Subject 9
Processing Heights
TRAIN: [ 1  2  3  4  6  8  9 10 12 13 15 16 18 20 22 23 25 26 28 29 31 32 33 34] TEST: [ 0  5  7 11 14 17 19 21 24 27 30]
TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 21 23 24 26 27 29 30 32 34] TEST: [ 1  3  6  9 12 15 18 22 25 28 31 33]
TRAIN: [ 0  1  3  5  6  7  9 11 12 14 15 17 18 19 21 22 24 25 27 28 30 31 33] TEST: [ 2  4  8 10 13 16 20 23 26 29 32 34]


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [   1    7   11   32   44   99  108  124  150  176  205  211  212  213
  214  215  219  220  225  226  228  230  231  235  238  240  249  250
  253  254  257  260  263  266  275  276  281  286  290  292  363  371
  408  424  430  584  648  674  694  743  780  787  788  799  800  805
  812  826  842  845  868  948  951  967  975 1021 1025 1055 1066 1095
 1101 1122 1129 1134 1143 1144 1154 1156 1162 1175 1176 1178 1190 1205
 1218 1354 1360 1386 1427 1450 1516 1764 1766 1770 1851 1916 1922 1950
 1957 1968 1970 2001 2003 2011 2026 2033 2041 2043 2073 2080 2091 2092
 2150 2245 2275 2279 2367 2368 2375 2468 2499 2538 2547 2548 2549 2568
 2602 2665 2719 2720 2741 2772 2780 2785 2816 2841 2852 2930 2966 2983
 2986 2996 3160 3219 3333 3390 3472 3475 3488 3505 3516 3551 3558 3620
 3705 3747 3753 3888 3891 3907 3911 3977 3984 4058 4060 4101 4110 4123


Scores: 0.4301 +- 0.1008
Chance Scores: 0.6190 +- 0.0949
Processing Social threat
TRAIN: [ 1  2  3  4  6  8  9 10 12 13 15 16 18 20 22 23 25 26 28 29 31 32 33 34] TEST: [ 0  5  7 11 14 17 19 21 24 27 30]
TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 21 23 24 26 27 29 30 32 34] TEST: [ 1  3  6  9 12 15 18 22 25 28 31 33]
TRAIN: [ 0  1  3  5  6  7  9 11 12 14 15 17 18 19 21 22 24 25 27 28 30 31 33] TEST: [ 2  4  8 10 13 16 20 23 26 29 32 34]
Scores: 0.5833 +- 0.1405
Chance Scores: 0.5387 +- 0.1948
Processing Spiders
TRAIN: [ 1  2  3  4  6  8  9 10 12 13 15 16 18 20 22 23 25 26 28 29 31 32 33 34] TEST: [ 0  5  7 11 14 17 19 21 24 27 30]
TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 21 23 24 26 27 29 30 32 34] TEST: [ 1  3  6  9 12 15 18 22 25 28 31 33]
TRAIN: [ 0  1  3  5  6  7  9 11 12 14 15 17 18 19 21 22 24 25 27 28 30 31 33] TEST: [ 2  4  8 10 13 16 20 23 26 29 32 34]


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [  11  137  143  150  154  163  170  176  178  187  193  230  234  236
  241  242  244  246  263  276  298  300  312  363  369  370  380  393
  395  454  572  651  674  700  743  767  768  781  791  792  794  798
  799  800  801  808  809  821  832  834  835  838  841  853  855  857
  921  948  953  954  959  971  974  975  978  979  983  984  986  988
  996 1006 1009 1014 1016 1017 1018 1021 1112 1156 1159 1165 1182 1197
 1202 1205 1231 1271 1356 1565 1605 1609 1646 1689 1690 1892 1952 1958
 1965 1973 1977 1979 1987 2005 2009 2020 2022 2067 2079 2080 2083 2151
 2159 2303 2401 2570 2751 2771 2780 2784 2798 2849 2910 2914 2943 2946
 2952 2956 2970 2972 2979 2981 2984 3039 3130 3154 3159 3184 3218 3418
 3488 3494 3548 3552 3555 3603 3701 3704 3753 3757 3805 3872 3939 3940
 4023 4031 4149 4205 4227 4263 4264 4331 4334 4338 4408 4430 4491 4495


Scores: 0.6285 +- 0.0299
Chance Scores: 0.3958 +- 0.0295
...
Overall Score: 0.5473 +- 0.1322
Overall Dummy Score: 0.5179 +- 0.1564


...
HTFA Summary Score Across Subjects, For Category Heights : 0.6706 +- 0.1895
HTFA Summary Score Across Subjects, For Category Social threat : 0.7071 +- 0.1921
HTFA Summary Score Across Subjects, For Category Spiders : 0.6835 +- 0.1958
...
HTFA Summary Score Across Subjects, Across Tasks: 0.6871 +- 0.1931
...
HTFA Summary Mean/CI, Across Tasks: 0.6847 +- 0.0631
